In [ ]:
# Need to satisfy following requirements:
# 1. pip instsall -r requirements.txt
# 2. pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI" 
# to address this issue: https://github.com/matterport/Mask_RCNN/issues/6
# also, you can specify this in reuirements.txt like this: https://stackoverflow.com/questions/16584552/how-to-state-in-requirements-txt-a-direct-github-source


## Training and predicting objects using Mask RCNN model

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Host pretrained model](#Host-pretrained-model)
4. [Train model](#Train-model)

## Introduction

placeholder

## Setup

Before preparing the data, there are some initial steps required for setup. To train the image classification algorithm on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To begin with, we need an AWS account role with SageMaker access. Here we will use the execution role the current notebook instance was given when it was created.  This role has necessary permissions, including access to your data in S3.

In [2]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::480604769687:role/service-role/AmazonSageMaker-ExecutionRole-20190530T172586


We also need to identify the S3 bucket that you want to use for providing training and validation datasets. It will be used to store the tranied model artifacts as well. In this notebook, we use a default bucket for use with SageMaker in your account. Alternatively, you could use whatever bucket you would like. We use an object prefix to help organize the bucket content.

In [4]:
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'DEMO-RCNN-model'

## Prepare dataset

## Instal Dependencies
See details here: https://github.com/matterport/Mask_RCNN



1. Clone this repository

2. Install dependencies
pip3 install -r requirements.txt

3. Run setup from the repository root directory
python3 setup.py install

4. Download pre-trained COCO weights (mask_rcnn_coco.h5) from the releases page.

5. (Optional) To train or test on MS COCO install pycocotools from one of these repos. They are forks of the original pycocotools with fixes for Python3 and Windows (the official repo doesn't seem to be active anymore).
Linux: https://github.com/waleedka/coco


In [5]:
! pygmentize code/setup.py

! git clone https://github.com/matterport/Mask_RCNN
! cd Mask_RCNN
! python setup.py install


## Inference of of pre-trained model